# Undistort Dataset

In [8]:
class Args:
    dataset_path = "../../resources/images/uncalibrated/calibration_dataset.npz"
    calibration_matrix_path = "../../resources/calibrations/calibration.yml"
args = Args()

In [5]:
import sys
sys.path.append("../../src")

In [6]:
from models.dataset.DatasetHelper import open_image_dataset_o3d, load_coefficients

## Load dataset

In [9]:
# Open Dataset and calibration file
rgbd_rs_images, rgbd_zv_images = open_image_dataset_o3d(args.dataset_path)
print(f"Opened dataset containing {len(rgbd_rs_images)} image sets")

Opened dataset containing 24 image sets


## Load coefficients

In [10]:
coefficients = load_coefficients(path=args.calibration_matrix_path)

## Transform Realsense ply to o3d mesh

In [13]:
import open3d as o3d

mesh = o3d.io.read_point_cloud("../../resources/images/uncalibrated/realsense/0.ply")
# o3d.visualization.draw_geometries([mesh])
undistored_mesh = mesh.transform(coefficients)
o3d.visualization.draw_geometries([mesh])

## Transform Zivid ply to o3d mesh

In [30]:
from matplotlib import pyplot as plt
import numpy as np
from zivid import PointCloud

def display_rgb(rgb, title="RGB image"):
    """Display RGB image.
    Args:
        rgb: RGB image (HxWx3 darray)
        title: Image title
    Returns None
    """
    plt.figure()
    plt.imshow(rgb)
    plt.title(title)
    plt.show(block=False)


def display_depthmap(xyz):
    """Create and display depthmap.
    Args:
        xyz: X, Y and Z images (point cloud co-ordinates)
    Returns None
    """
    plt.figure()
    plt.imshow(
        xyz[:, :, 2],
        vmin=np.nanmin(xyz[:, :, 2]),
        vmax=np.nanmax(xyz[:, :, 2]),
        cmap="viridis",
    )
    plt.colorbar()
    plt.title("Depth map")
    plt.show(block=False)


def display_pointcloud(xyz, rgb, normals=None):
    """Display point cloud provided from 'xyz' with colors from 'rgb'.
    Args:
        rgb: RGB image
        xyz: X, Y and Z images (point cloud co-ordinates)
        normals: ordered array of normal vectors, mapped to xyz
    Returns None
    """
    xyz = np.nan_to_num(xyz).reshape(-1, 3)
    rgb = rgb.reshape(-1, 3)

    point_cloud_open3d = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(xyz))
    point_cloud_open3d.colors = o3d.utility.Vector3dVector(rgb / 255)
    if normals is not None:
        normals = np.nan_to_num(normals).reshape(-1, 3)
        point_cloud_open3d.normals = o3d.utility.Vector3dVector(normals)
        print("Open 3D controls:")
        print("  n: for normals")
        print("  9: for point cloud colored by normals")
        print("  h: for all controls")

    return point_cloud_open3d

    visualizer = o3d.visualization.Visualizer()  # pylint: disable=no-member
    visualizer.create_window()
    visualizer.add_geometry(point_cloud_open3d)

    if normals is None:
        visualizer.get_render_option().background_color = (0, 0, 0)
    visualizer.get_render_option().point_size = 1
    visualizer.get_render_option().show_coordinate_frame = True
    visualizer.get_view_control().set_front([0, 0, -1])
    visualizer.get_view_control().set_up([0, -1, 0])

    visualizer.run()
    visualizer.destroy_window()


def display_pointcloud_with_downsampled_normals(point_cloud: PointCloud, downsampling: PointCloud.Downsampling):
    """Display point cloud with downsampled normals
    Args:
        point_cloud: a Zivid point cloud handle
        downsampling: a valid Zivid downsampling factor to apply to normals
    Returns None
    """
    rgb = point_cloud.copy_data("rgba")[:, :, :3]
    xyz = point_cloud.copy_data("xyz")
    point_cloud.downsample(downsampling)
    normals = point_cloud.copy_data("normals")

    display_pointcloud(xyz=xyz, rgb=rgb, normals=normals)

In [34]:
import zivid

app = zivid.Application()
filename_zdf = "../../resources/images/uncalibrated/zivid/6.zdf"
frame = zivid.Frame(filename_zdf)
point_cloud = frame.point_cloud()
xyz = point_cloud.copy_data("xyz")
rgba = point_cloud.copy_data("rgba")

# display_rgb(rgba[:, :, 0:3])

# display_depthmap(xyz)

zivid_pointcloud = display_pointcloud(xyz, rgba[:, :, 0:3])
undistored_mesh = zivid_pointcloud.transform(coefficients)
visualizer = o3d.visualization.Visualizer()  # pylint: disable=no-member
visualizer.create_window()
visualizer.add_geometry(undistored_mesh)

visualizer.get_render_option().background_color = (0, 0, 0)
visualizer.get_render_option().point_size = 1
visualizer.get_render_option().show_coordinate_frame = True
visualizer.get_view_control().set_front([0, 0, -1])
visualizer.get_view_control().set_up([0, -1, 0])

visualizer.run()
visualizer.destroy_window()

In [17]:
import open3d as o3d
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()
cam = o3d.camera.PinholeCameraIntrinsic()
cam.intrinsic_matrix = [[500, 0.0, 0.0], [0.0, 918.8529534152894, 0.0], [959.5, 530.0, 1.0]]

for rgbd_rs_image in rgbd_rs_images:
    pc_rs = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_rs_image, cam)
    o3d.visualization.draw_geometries([pc_rs])

KeyboardInterrupt: 